This notebook is part of [**byron v0.1**](https://github.com/microgp/byron)  
Copyright 2023 Giovanni Squillero and Alberto Tonda  
SPDX-License-Identifier: Apache-2.0  

In [1]:
import logging
import math

logging.basicConfig(format="[%(asctime)s] %(levelname)s:%(message)s", datefmt="%H:%M:%S", level=logging.DEBUG)

In [2]:
try:
    import byron
except ModuleNotFoundError as e:
    import sys, os

    logging.debug("import: Failed to load byron (%s), adding ../src directory.", e)
    sys.path.append(os.path.join(os.pardir, "src"))
    import byron

[10:24:19] DEBUG:import: Failed to load byron (No module named 'byron'), adding ../src directory.


# Fitness Demo

`Fitness` provides a consistent framework to store and compare the evaluations of individual. Given two individuals $i_1$ and $i_2$ with fitness values, respectively, equal to `f1` and `f2`:

* if `f1 > f2`, then $i_1$ is more fit than $i_2$ (i.e., it is preferable)
* if `f1 < f2`, then $i_2$ is more fit than $i_1$ (i.e., it is preferable)
* if `f1 == f2`, then $i_1$ and $i_2$ may be considered equivalent

## Scalar Fitness

`Scalar` fitness are floating-point, single-value fitness. A high fitness value is better than a low one. Rounding is taken into consideration: a fitness value of $(\sqrt{2})^2$ is considered to be equivalent to a fitness value of $2$, although in Python `math.sqrt(2) ** 2 != 2` due to approximations.

In [3]:
v1 = 2
v2 = math.sqrt(2) ** 2

f1 = byron.fit.Scalar(v1)
f2 = byron.fit.Scalar(v2)

print(f"{v1} == {v2} is {v1 == v2}  /  {v1} < {v2} is {v1 < v2}")
print(f"{f1} == {f2} is {f1 == f2}  /  {f1} < {f2} is {f1 < f2}")

2 == 2.0000000000000004 is False  /  2 < 2.0000000000000004 is True
⸨2⸩ == ⸨2⸩ is True  /  ⸨2⸩ < ⸨2⸩ is False


The class `ScalarExact` handles floating point without rounding.

In [4]:
v1 = 2
v2 = math.sqrt(2) ** 2

f1 = byron.fit.ScalarExact(v1)
f2 = byron.fit.ScalarExact(v2)

print(f"{v1} == {v2} is {v1 == v2}  /  {v1} < {v2} is {v1 < v2}")
print(f"{f1} == {f2} is {f1 == f2}  /  {f1} < {f2} is {f1 < f2}")

2 == 2.0000000000000004 is False  /  2 < 2.0000000000000004 is True
⸨2.0♯⸩ == ⸨2.0000000000000004♯⸩ is False  /  ⸨2.0♯⸩ < ⸨2.0000000000000004♯⸩ is True


The class `ScalarInteger` handles integer values. Large numbers as $17^{19}$ and $17^{19}+1$ can be used, while a `Scalar` fitness would suffer from rounding.

In [5]:
v1 = 17**19
v2 = v1 + 1

fi1 = byron.fit.ScalarInteger(v1)
fi2 = byron.fit.ScalarInteger(v2)
print(f"{fi1} == {fi2} is {fi1 == fi2}  /  {fi1} < {fi2} is {fi1 < fi2}")

fs1 = byron.fit.Scalar(v1)
fs2 = byron.fit.Scalar(v2)
print(f"{fs1} == {fs2} is {fs1 == fs2}  /  {fs1} < {fs2} is {fs1 < fs2}")

⸨239,072,435,685,151,324,847,153𝕚⸩ == ⸨239,072,435,685,151,324,847,154𝕚⸩ is False  /  ⸨239,072,435,685,151,324,847,153𝕚⸩ < ⸨239,072,435,685,151,324,847,154𝕚⸩ is True
⸨2.39072e+23⸩ == ⸨2.39072e+23⸩ is True  /  ⸨2.39072e+23⸩ < ⸨2.39072e+23⸩ is False


To minimize instead of maximizing the value (i.e., smaller is fitter), all `Fitness` can be reversed.

In [6]:
MyFitnessR = byron.fit.reverse_fitness(byron.fitness.Scalar)

f1 = MyFitnessR(23)
f2 = MyFitnessR(10)

print(f"{f1} == {f2} is {f1 == f2}  /  {f1} > {f2} is {f1 > f2}  /  {f1} < {f2} is {f1 < f2}")

⸨ᴙ23⸩ == ⸨ᴙ10⸩ is False  /  ⸨ᴙ23⸩ > ⸨ᴙ10⸩ is False  /  ⸨ᴙ23⸩ < ⸨ᴙ10⸩ is True


Reversed fitnesses may be reversed...

In [7]:
MyFitnessR = byron.fit.reverse_fitness(byron.fitness.Scalar)
MyFitnessRR = byron.fit.reverse_fitness(MyFitnessR)

f1 = MyFitnessRR(23)
f2 = MyFitnessRR(10)

print(f"{f1} == {f2} is {f1 == f2}  /  {f1} > {f2} is {f1 > f2}  /  {f1} < {f2} is {f1 < f2}")

⸨ᴙᴙ23⸩ == ⸨ᴙᴙ10⸩ is False  /  ⸨ᴙᴙ23⸩ > ⸨ᴙᴙ10⸩ is True  /  ⸨ᴙᴙ23⸩ < ⸨ᴙᴙ10⸩ is False


In [8]:
f1 = byron.fit.Vector([23, 10])
f2 = byron.fit.Vector([18, 5])

print(f"{f1} == {f2} is {f1 == f2}  /  {f1} < {f2} is {f1 < f2}  /  {f1} > {f2} is {f1 > f2}")

⟦23.0, 10.0⟧ == ⟦18.0, 5.0⟧ is False  /  ⟦23.0, 10.0⟧ < ⟦18.0, 5.0⟧ is False  /  ⟦23.0, 10.0⟧ > ⟦18.0, 5.0⟧ is True


In [9]:
f1 = byron.fit.Sequence(
    [
        byron.fit.ScalarInteger(0),
        byron.fit.ScalarInteger(1),
        byron.fit.ScalarInteger(2),
        byron.fit.ScalarInteger(3),
        byron.fit.ScalarInteger(0),
    ]
)
f2 = byron.fit.Sequence(
    [
        byron.fit.ScalarInteger(0),
        byron.fit.ScalarInteger(3),
        byron.fit.ScalarInteger(2),
        byron.fit.ScalarInteger(1),
        byron.fit.ScalarInteger(0),
    ]
)

print(f"{f1} == {f2} is {f1 == f2}  /  {f1} < {f2} is {f1 < f2}  /  {f1} > {f2} is {f1 > f2}")

⸨0𝕚, 1𝕚, 2𝕚, 3𝕚, 0𝕚⸩ == ⸨0𝕚, 3𝕚, 2𝕚, 1𝕚, 0𝕚⸩ is False  /  ⸨0𝕚, 1𝕚, 2𝕚, 3𝕚, 0𝕚⸩ < ⸨0𝕚, 3𝕚, 2𝕚, 1𝕚, 0𝕚⸩ is True  /  ⸨0𝕚, 1𝕚, 2𝕚, 3𝕚, 0𝕚⸩ > ⸨0𝕚, 3𝕚, 2𝕚, 1𝕚, 0𝕚⸩ is False
